In [1]:
import glob
import numpy as np
import pandas as pd
import pickle
import pprint

from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from joblib import parallel_backend


proj_dir = '/home/drumblum/Northeastern/cs6140/CS6140-final-project/'
raw_dir = proj_dir + "raw/"
res_dir = proj_dir + "res/"
glob_2018 = "Flights_2018_*.csv"
single_2018 = "Flights_2018_1.csv"
pickle_file = "2018_df.pkl"

In [5]:
''' Cell to get a single CSV file loaded into DF
'''
# df = pd.read_csv(raw_dir + single_2018, low_memory=False)

In [35]:
''' Cell to load ALL 2018 CSV files into DF
'''

# csv_files = glob.glob(str(raw_dir) + "/Flights_2018_*.csv")
# csv_files = glob.glob(raw_dir + glob_2018)
# print(csv_files)
# def read_csv_low_memory(csv):
#     return pd.read_csv(csv, low_memory=False)

# df = pd.concat(map(read_csv_low_memory, csv_files))
# df.to_pickle(res_dir + pickle_file)


In [3]:
'''Cell to get the pickedl ALL 2018 into DF
'''
df = pd.read_pickle(res_dir + pickle_file)

In [6]:
def analyze_model(model, X_test, X_train, y_test, y_train):
    # TODO score vs score_samples
    test_accuracy = model.score(X_test, y_test)
    train_accuracy = model.score(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    prf = precision_recall_fscore_support(y_test, y_pred, average='binary') # TODO different average values: micro macro binary weighted samples

    print(f"Test accuracy: {test_accuracy}")
    print(f"Train accuracy: {train_accuracy}")
    print(f"Precision: {prf[0]}")
    print(f"Recall: {prf[1]}")
    print(f"F-Beta Score: {prf[2]}")
    print(f"F1 Score: {f1}")

    ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    plt.show()

In [7]:
df_modified = df.drop(columns=[
                               'Div1Airport', 
                               'Div1AirportID', 
                               'Div1AirportSeqID', 
                               'Div1WheelsOn', 
                               'Div1TotalGTime', 
                               'Div1LongestGTime', 
                               'Div1WheelsOff', 
                               'Div1TailNum',
                               'Div2Airport', 
                               'Div2AirportID', 
                               'Div2AirportSeqID', 
                               'Div2WheelsOn', 
                               'Div2TotalGTime', 
                               'Div2LongestGTime', 
                               'Div2WheelsOff', 
                               'Div2TailNum',
                               'Div3Airport', 
                               'Div3AirportID', 
                               'Div3AirportSeqID', 
                               'Div3WheelsOn', 
                               'Div3TotalGTime', 
                               'Div3LongestGTime', 
                               'Div3WheelsOff', 
                               'Div3TailNum',
                               'Div4Airport', 
                               'Div4AirportID', 
                               'Div4AirportSeqID', 
                               'Div4WheelsOn', 
                               'Div4TotalGTime', 
                               'Div4LongestGTime', 
                               'Div4WheelsOff', 
                               'Div4TailNum',
                               'Div5Airport', 
                               'Div5AirportID', 
                               'Div5AirportSeqID', 
                               'Div5WheelsOn', 
                               'Div5TotalGTime', 
                               'Div5LongestGTime', 
                               'Div5WheelsOff', 
                               'Div5TailNum', 
                               'Duplicate', 
                               'Unnamed: 119'])
# df_modified = df_modified[df_modified['ArrDel15']]
df_modified = df_modified.dropna(subset=['ArrDel15'])

In [14]:
df_modified['Dest']

0         ATL
1         ATL
2         ATL
3         ATL
4         ATL
         ... 
350039    ORD
350040    MKE
350041    ORD
350042    BNA
350043    IAD
Name: Dest, Length: 5586619, dtype: object

In [20]:
# df_modified[df_modified['ArrDel15'].isnull()]
df_boston = df_modified.query('Dest == "BOS"')


In [21]:
# df[(df['ArrDel15'] != math.isnan)]
# df = df[["DayofMonth","Month","Year", "Distance","DepDelay", "ArrDel15"]].dropna()# get rid of nan
print(df_boston.ArrDel15.unique()) # find unique values

flight_set =  df_boston[["DayofMonth","Month","Year", "Distance", "DepDelay", "ArrDel15"]]
flight_set = flight_set.dropna()
X = flight_set[["DayofMonth","Month","Year", "Distance", "DepDelay"]]
# X = X.dropna()
# X = X.drop(columns=['ArrDel15'])
# X = df_modified.drop(columns=['ArrDel15', 'FlightDate'])
Y = flight_set[["ArrDel15"]]
Y = Y.ArrDel15.ravel() # flatten
print(Y.shape)

# split into train and 
x_train, x_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=150) # TODO how to choose random state

[1. 0.]
(112434,)


In [22]:
# log_reg = LogisticRegression(n_jobs=20)
log_reg = LogisticRegression()

param_grid = [
  # {'solver':['liblinear'],'C':[.001, .01, .5, .1, 1, 5, 10], 'penalty':['l1','l2'], 'n_jobs': [2]},
  # {'solver':['saga'],'C':[.001, .01, .5, .1, 1, 5, 10], 'penalty':['l2','elasticnet'], 'n_jobs': [-1]} # l1 gives erros here
  # {'solver':['saga'],'C':[.001, .01, .5, .1, 1, 5, 10], 'penalty':['l2','elasticnet'], 'n_jobs': [-1]} # l1 gives erros here
  # {'solver':['saga'],'C':[.001, .01], 'penalty':['l2']} 

  {'solver':['liblinear'],'C':[.001, .01, .5, .1, 1, 5, 10], 'penalty':['l1'], 'n_jobs': [1]},
  {'solver':['saga'],'C':[.001, .01, .5, .1, 1, 5, 10], 'penalty':['l2', 'elasticnet'], 'n_jobs': [-1] } 
]

# TODO n_jobs = 5 is ideal for my computer, results may vary
# TODO choose best scoring function f1, f1_macro, etc.
search = GridSearchCV(log_reg, param_grid, scoring='f1', cv=5, verbose=1) 
# TODO how to choose cv
# TODO choose verbosity
# TODO use scoring param?



In [23]:

logreg_model = search.fit(x_train, y_train)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/drumblum/mambaforge/envs/cs6140/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/drumblum/mambaforge/envs/cs6140/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/drumblum/mambaforge/envs/cs6140/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/drumblum/mambaforge/envs/cs6140/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/drumblum/mambaforge/envs/cs6140/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/dr

In [45]:
# pprint.pprint(logreg_model.best_estimator_.get_params())
# pprint.pprint(logreg_model.cv_results_)
pickle_model = pickle.dumps(search)
# f = open(res_dir + 'model-pickle.pkl', 'wb')
# pickle.dump(pickle_model, f)
with open(res_dir + 'model-pickle.pkl', 'wb') as f:
#     # f.write('hello\n')
    pickle.dump(pickle_model, f)